In [ ]:
!pip install spectral
import tensorflow as tf
import spectral
import spectral.io.envi as envi
import os
import pathlib
import time
import datetime
from google.colab import drive

from matplotlib import pyplot as plt
from IPython import display

In [ ]:
import numpy as np
import cv2

rgb_images = []  
hsi_images = []

# Mount your Google Drive
drive.mount('/content/drive')

# Set the directory path for the hyperspectral images in your Google Drive
dir_path = '/content/drive/MyDrive'

stop = False

for dir_name in os.listdir(dir_path):
    if dir_name not in ["HSI_L", "HSI_N", "HSI_P"]:
        continue

    if (stop):
      break

    print(f'Processing images in directory {dir_name}...')

    if os.path.isdir(os.path.join(dir_path, dir_name)):
        # Get the corresponding RGB directory name
        rgb_dir_name = dir_name.replace('HSI', 'RGB')

        # Loop over all image files in the directory
        for file_name in os.listdir(os.path.join(dir_path, dir_name)):
            if (len(rgb_images)==2):
              stop = True
              break

            if file_name.endswith('.hdr'):
                # Load the HSI image file
                hsi_file_path = os.path.join(dir_path, dir_name, file_name)
                hsi_image = envi.open(hsi_file_path, hsi_file_path[:-4] + '.raw')

                # Preprocess the HSI image
                hsi_data = hsi_image.load()
                height, width = hsi_image.shape[:2]
                hsi_data = hsi_data.reshape(height, width, -1)
                hsi_data_min = np.min(hsi_data)
                hsi_data_max = np.max(hsi_data)
                hsi_data = (hsi_data - hsi_data_min) / (hsi_data_max - hsi_data_min)

                # Load the corresponding RGB image file
                rgb_file_name = os.path.splitext(file_name)[0] + '.jpg'
                rgb_file_path = os.path.join(dir_path, rgb_dir_name, rgb_file_name)
                if not os.path.exists(rgb_file_path):
                  rgb_file_name = rgb_file_name[:6] + 'C' + rgb_file_name[7:]
                  rgb_file_path = os.path.join(dir_path, rgb_dir_name, rgb_file_name)
                
                if not os.path.exists(rgb_file_path):
                  continue
                  
                rgb_image = cv2.imread(rgb_file_path)
                print(rgb_dir_name)
                print(rgb_file_name)

                # Preprocess the RGB image
                rgb_data = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)
                rgb_data = rgb_data.astype(np.float32) / 255.0

                # Append the preprocessed HSI and RGB images to their respective lists
                hsi_images.append(hsi_data)
                rgb_images.append(rgb_data)

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential

def generator(input_shape=(1728, 2304, 3)):
    model = Sequential()

    # Layer 1
    model.add(Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))

    # Layer 2
    model.add(Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Layer 3
    model.add(Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Layer 4
    model.add(Conv2DTranspose(256, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Layer 5
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Layer 6
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Layer 7
    model.add(Conv2DTranspose(60, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(60, kernel_size=4, strides=2, padding='same', activation='tanh'))

    return model

In [ ]:
generator = generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential

def discriminator(input_shape=(1024, 1280, 60)):
    model = Sequential()

    # Layer 1
    model.add(Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))

    # Layer 2
    model.add(Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Layer 3
    model.add(Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Layer 4
    model.add(Conv2D(512, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [ ]:
discriminator = discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = loss_object(tf.ones_like(real_output), real_output)
    fake_loss = loss_object(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return loss_object(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
@tf.function
def train_step(rgb_images, hsi_images):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(rgb_images, training=True)

        real_output = discriminator(hsi_images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, disc_loss

In [ ]:
def train(rgb_dataset, hsi_dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for rgb_images, hsi_images in zip(rgb_dataset, hsi_dataset):
            gen_loss, disc_loss = train_step(rgb_images, hsi_images)

       

        print(f'Epoch {epoch + 1}, gen_loss={gen_loss:.4f}, disc_loss={disc_loss:.4f}, time={time.time() - start:.2f} sec')


In [ ]:
# Train the model
EPOCHS = 3
rgb_images2 = np.array(rgb_images)
hsi_images2 = np.array(hsi_images)

train(rgb_images2, hsi_images2, EPOCHS)